In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11267,2024-05-25,Chile Lnb 2,20:00,Municipal Chillan,Aleman,2.68,1.43,138.5,1.85,1.85,0.0,0.00,0.00,fHMeYE9b,0.373134,0.699301,0.540541,0.540541,0.072435,323.200,349.659254,1.081867,1.2,1.643168,1.369306,141.52,4.572,4.779887,1.045470,-13.0,176.140,113.342226,0.643478,1.8,1.643168,0.912871,118.56,2.256,1.361518,0.603510,3.0,61,76,2.32,1.56,0.000,0.000,0.430114,0.000000,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11268,2024-05-25,Chile Lnb 2,20:15,Colo Colo,Stadio Italiano,1.04,9.20,141.5,1.85,1.85,0.0,0.00,0.00,GQNiZffh,0.961538,0.108696,0.540541,0.540541,0.070234,107.892,26.255193,0.243347,3.0,0.000000,0.000000,130.50,1.274,0.276821,0.217285,91.0,109.260,35.854588,0.328158,2.4,1.341641,0.559017,150.72,1.556,0.532241,0.342057,52.0,87,96,1.50,1.57,0.000,0.000,1.126951,0.000000,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11269,2024-05-25,Chile Lnb 2,21:30,Luis Matte Larrain,Brisas,1.16,4.75,141.5,1.85,1.85,0.0,0.00,0.00,4nfZGDvO,0.862069,0.210526,0.540541,0.540541,0.072595,183.610,124.847944,0.679963,1.2,1.643168,1.369306,213.84,2.674,1.547750,0.578815,-8.0,360.764,158.154327,0.438387,0.6,1.341641,2.236068,176.32,5.436,2.530263,0.465464,-28.0,72,76,2.97,2.32,0.000,0.000,0.859057,0.000000,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11270,2024-05-25,Chile Lnb 2,21:30,Sergio Ceppi,Boston C. C.,1.64,2.18,143.5,1.85,1.85,0.0,0.00,0.00,rwgwGXgU,0.609756,0.458716,0.540541,0.540541,0.068472,115.290,42.374140,0.367544,1.2,1.643168,1.369306,171.12,1.546,0.583507,0.377430,0.0,256.398,157.402719,0.613900,2.4,1.341641,0.559017,84.80,3.800,2.871872,0.755756,24.0,69,80,2.48,1.06,0.000,0.000,0.199915,0.000000,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11271,2024-05-25,Estônia Korvpalli Meistriliiga,11:00,Rapla,Pärnu,3.12,1.33,141.5,1.85,1.83,-2.5,2.02,1.67,IcQJypOc,0.320513,0.751880,0.540541,0.546448,0.072393,519.732,622.502564,1.197738,1.2,1.643168,1.369306,172.04,8.746,11.214260,1.282216,-73.0,153.214,76.507400,0.499350,1.2,1.643168,1.369306,96.20,2.256,0.851135,0.377276,-7.0,68,65,2.53,1.48,0.000,188.118,0.568863,0.007686,0.134139,0.00,0.000,inf,0.0,0.0,0.0,0.93,0.186,1.774194,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,2024-05-25,Austrália Nbl1 Norte,05:30,Cairns F,Logan Thunder F,2.97,1.36,156.5,1.87,1.83,0.0,0.00,0.00,MBhNLb6C,0.336700,0.735294,0.534759,0.546448,0.071994,1370.816,745.376327,0.543746,0.0,0.000000,NaN,1023.00,23.062,16.685899,0.723523,-148.0,278.590,148.014302,0.531298,1.2,1.643168,1.369306,143.36,4.014,2.409270,0.600217,-20.0,93,64,11.00,2.24,0.000,0.000,0.525839,0.015289,NaN,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11379,2024-05-25,Austrália Nbl1 Norte,04:00,Northside Wizards F,Southern District Spartans F,1.01,28.00,149.5,1.85,1.85,0.0,0.00,0.00,lbfFNKy0,0.990099,0.035714,0.540541,0.540541,0.025813,745.194,1431.568881,1.921069,3.0,0.000000,0.000000,3306.00,6.632,12.504200,1.885434,242.0,97.318,29.582189,0.303974,1.8,1.643168,0.912871,147.20,1.238,0.223540,0.180565,46.0,114,92,29.00,1.60,0.000,0.000,1.315740,0.000000,NaN,0.0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11294,15:20,Israel Superliga,Hapoel Jerusalem,Hapoel Tel-Aviv,2.03,Back Home
11309,08:00,Rússia Liga Vtb United,Zenit Petersburg,Lokomotiv Kuban,1.78,Back Home
11372,06:30,Austrália Nbl1 Sul,Frankston,Sandringham Sabres,1.88,Back Home
